In [20]:
# coding: utf-8

import pandas as pd
from impala.dbapi import connect
from impala.util import as_pandas
import ast
import math

def agg_by_weekindex_sales_volumn(unit_totalamount):
    
    conn = connect(host='salest-master-server', port=21050)
    cur = conn.cursor()
        
    start_date = "2014/09/01"
    end_date = "2016/04/30"
    
    target_date_idx = pd.date_range(start_date,end_date, freq='WS')
        
    target_date_arr = target_date_idx.strftime('%Y-%m-%d')
    target_date_tuple = tuple(target_date_arr)
    
    conn.close()
    
    return target_date_tuple


agg_by_weekindex_sales_volumn(1000)

ValueError: Could not evaluate WS